In [1]:
use "..\Data\cleaned_cps"

replace experience =round(experience,1)
gen fulltime_fullyear = fulltime*fullyear // focus on full time full year workers

/* replace wages with missing values for non fulltime full year workers or bottom weekly wages */ 
replace weekly_wage  = . if !fulltime_fullyear | weekly_wage_less_than_sixtyseven



(1,893,739 real changes made)


(1,231,785 real changes made, 1,231,785 to missing)


In [2]:
/* Generate real weekly wages again */
summarize pce if inlist(year, 2001), meanonly // 2001 survey year = 2000 pce year
scalar pce2000=r(mean)
scalar list pce2000

gen real_wage = (pce2000/pce) * weekly_wage
sum real_wage




   pce2000 =     78.235

(1,655,747 missing values generated)


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
   real_wage |  2,733,272    724.2206    588.4449   110.4579   10107.46


In [3]:
/* We create weights */
gen n_obs = 1
label variable n_obs "Number of observations"

gen n_weight = asecwt
label variable n_weight "Earnings weight"

gen n_labor_supply_weight = asecwt*wkswork1
label variable n_labor_supply_weight "Earnings weight times weeks last year"

gen n_labor_supply_hourly_weight = asecwt*wkswork1*weekly_hours_workedly
label variable n_labor_supply_hourly_weight "Earnings weight times weeks last year times hours last year"


/* We assign weights for missing values of weekly wages */
gen m_obs = 1
replace m_obs = 0 if weekly_wage  == . | allocated_earnings == 1 | selfemployed==1
label variable m_obs "Number of FTFY obs with non-missing earnings"

gen m_weight = asecwt
replace m_weight = 0  if weekly_wage  == . | allocated_earnings == 1 | selfemployed==1
label variable m_weight "Earnings weight for FTFY obs with non-missing earnings"

gen m_labor_supply_weight = asecwt*wkswork1
replace m_labor_supply_weight = 0 if weekly_wage  == . | allocated_earnings == 1 | selfemployed==1
label variable m_labor_supply_weight "Earnings weight times weeks last year for FTFY obs with non-missing earnings"











(2,082,384 real changes made)



(2,082,317 real changes made)



(2,082,317 real changes made)



In [4]:
save precollapse, replace


(note: file precollapse.dta not found)
file precollapse.dta saved


In [5]:
collapse (sum) n_obs n_weight n_labor_supply_weight n_labor_supply_hourly_weight m_obs m_weight m_labor_supply_weight, by(year schooling experience female)
sort year schooling experience female

In [6]:
save aftercollapse, replace

(note: file aftercollapse.dta not found)
file aftercollapse.dta saved


In [7]:
use precollapse 


In [8]:
collapse (mean) real_wage weekly_wage [aweight=m_weight], by(year schooling experience female)
sort year schooling experience female

In [9]:
merge m:1 year schooling experience female using "../Relative Supply/aftercollapse", keep(match) nogenerate
sum
save aftercollapse_merged, replace



    Result                           # of obs.
    -----------------------------------------
    not matched                             0
    matched                            26,264  
    -----------------------------------------


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
        year |     26,264    1991.762     16.4486       1964       2020
      female |     26,264    .4982866    .5000066          0          1
  experience |     26,264    22.78552     13.7232          0         58
   schooling |     26,264    2.877056    1.417441          1          5
   real_wage |     26,264     717.161    357.1733   115.8597   5898.612
-------------+---------------------------------------------------------
 weekly_wage |     26,264    593.1871     485.493    28.2335   7769.038
       n_obs |     26,264    167.0871    144.5288          1       2021
    n_weight |     26,264    265790.1    22

In [10]:
keep if experience>=0 & experience<=39

gen experience_cat = 1
replace experience_cat = 2 if experience>=10 & experience<=19
replace experience_cat = 3 if experience>=20 & experience<=29
replace experience_cat = 4 if experience>=30 & experience<=39



(3,484 observations deleted)


(5,696 real changes made)

(5,700 real changes made)

(5,693 real changes made)


In [11]:
egen basewage = max(real_wage*(female==0)*(schooling==2)*(experience==10)), by(year) // yearly wage of H.S graduate with 10 yrs experience
tab year, sum(basewage)
gen normalized_wage = (real_wage/basewage)
tab year, sum(normalized_wage)
egen cell_efficieny_unit=mean(normalized_wage), by(schooling female experience) 
tab year, sum(cell_efficieny_unit)




            |         Summary of basewage
survey year |        Mean   Std. Dev.       Freq.
------------+------------------------------------
       1964 |   518.85175           0         393
       1965 |   548.75403           0         395
       1966 |   563.52856           0         399
       1967 |   597.06323           0         396
       1968 |   601.66254           0         399
       1969 |   597.23743           0         400
       1970 |   641.94122           0         400
       1971 |   633.20251           0         400
       1972 |   654.49268           0         400
       1973 |    672.4082           0         400
       1974 |   677.25848           0         400
       1975 |   679.01947           0         400
       1976 |   626.73419           0         400
       1977 |   638.47723           0         400
       1978 |   658.44513           0         400
       1979 |   682.68701           0         399
       1980 |   644.20642           0         400
     

       2001 |   1.2309558   .55376348         400
       2002 |   1.2309558   .55376348         400
       2003 |   1.2309558   .55376348         400
       2004 |   1.2309558   .55376348         400
       2005 |   1.2309558   .55376348         400
       2006 |   1.2309558   .55376348         400
       2007 |   1.2309558   .55376348         400
       2008 |   1.2309558   .55376348         400
       2009 |   1.2309558   .55376348         400
       2010 |   1.2309558   .55376348         400
       2011 |   1.2309558   .55376348         400
       2012 |   1.2309558   .55376348         400
       2013 |   1.2309558   .55376348         400
       2014 |   1.2309558   .55376348         400
       2015 |   1.2309558   .55376348         400
       2016 |   1.2309558   .55376348         400
       2017 |   1.2309558   .55376348         400
       2018 |   1.2309558   .55376348         400
       2019 |   1.2309558   .55376348         400
       2020 |   1.2309558   .55376348         400


In [12]:
egen tot_efficiency_unit_weighted = sum(cell_efficieny_unit*n_labor_supply_hourly_weight),by(year)
gen efficiency_unit_weighted=((cell_efficieny_unit)*(n_labor_supply_hourly_weight)/tot_efficiency_unit_weighted)

egen college_supply = sum(efficiency_unit_weighted*((schooling==4 | schooling==5) + 0.5*(schooling ==3))),by(year)

gen college_hs_supply=ln(college_supply/(1-college_supply))

In [13]:
table year, c(mean college_hs_supply)


--------------------------
survey    |
year      | mean(colleg~y)
----------+---------------
     1964 |      -.7918856
     1965 |      -.7444239
     1966 |      -.7511503
     1967 |      -.7342879
     1968 |       -.671908
     1969 |      -.6682243
     1970 |       -.624491
     1971 |      -.5646536
     1972 |      -.5354807
     1973 |      -.4951653
     1974 |      -.4356841
     1975 |      -.3676135
     1976 |      -.3160161
     1977 |      -.2845219
     1978 |       -.261696
     1979 |      -.2226935
     1980 |      -.1967725
     1981 |      -.1667217
     1982 |      -.1223423
     1983 |      -.0274082
     1984 |      -.0050444
     1985 |      -.0067089
     1986 |      -.0000794
     1987 |       .0218796
     1988 |       .0257907
     1989 |       .0646573
     1990 |       .0632647
     1991 |       .0729961
     1992 |       .0975208
     1993 |       .1473078
     1994 |       .1690533
     1995 |        .199798
     1996 |       .1895386
     1997 |    

In [16]:
collapse college_hs_supply, by(year)
save "../Data/relative_supply", replace 